# Tutorial 15: FastMCP Server Basics

## Building Custom MCP Servers for Enterprise AI Agents

### What You'll Learn

In this tutorial, you'll learn how to build a **FastMCP server** that exposes custom tools to AI agents using the Model Context Protocol (MCP). This is the foundation for building enterprise-grade, scalable AI agent architectures.

**Key Concepts:**
- Model Context Protocol (MCP) overview
- Creating a FastMCP server
- Defining custom tools with schemas
- Testing MCP servers locally
- Integrating MCP servers with AI agents

**Use Case:** Building a Travel Booking MCP Server with flight search, hotel availability, and currency conversion tools.

**Duration:** 45 minutes

---

### Prerequisites

- Completed tutorials 01-14
- Python 3.11+
- Understanding of REST APIs
- Azure OpenAI or OpenAI API access

## Part 1: Understanding Model Context Protocol (MCP)

### What is MCP?

**Model Context Protocol (MCP)** is an open standard for connecting AI assistants to external data sources and tools. Think of it as a standardized way for AI agents to:

- Access external data (databases, APIs, file systems)
- Execute functions and tools
- Maintain context across interactions
- Scale agent capabilities without modifying agent code

### Why FastMCP?

**FastMCP** is a Python framework that makes it easy to build MCP servers with:
- Simple, decorator-based tool definitions
- Automatic schema generation
- Built-in validation
- Easy local testing
- Production-ready server implementation

### Architecture

```
┌─────────────────┐
│   AI Agent      │
│  (OpenAI/Azure) │
└────────┬────────┘
         │ MCP Protocol
         │
┌────────┴────────┐
│  FastMCP Server │
│  ┌───────────┐  │
│  │  Tool 1   │  │
│  │  Tool 2   │  │
│  │  Tool 3   │  │
│  └───────────┘  │
└─────────────────┘
```

## Part 2: Setup and Installation

First, let's install the required packages for building our FastMCP server.

In [5]:
# Install FastMCP and required dependencies
# Using latest compatible versions
%pip install fastmcp uvicorn python-dotenv

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [17]:
# Import required libraries
import os
from datetime import datetime, timedelta
from typing import Optional, List
from pydantic import BaseModel, Field
from dotenv import load_dotenv
import json

# Load environment variables
load_dotenv()

print("Environment loaded successfully")

Environment loaded successfully


## Part 3: Creating Your First FastMCP Server

Let's build a Travel Booking MCP Server with three tools:
1. **search_flights** - Search for available flights
2. **check_hotel_availability** - Check hotel room availability
3. **convert_currency** - Convert between currencies

We'll create a file called `travel_mcp_server.py` that contains our MCP server implementation.

In [25]:
# Create the MCP server file
server_code = '''
"""
Travel Booking MCP Server
A FastMCP server that provides travel-related tools for AI agents
"""

from fastmcp import FastMCP
from datetime import datetime, timedelta
from typing import List, Optional, Dict
from pydantic import BaseModel, Field
import random

# Initialize the MCP server
mcp = FastMCP("Travel Booking Server")

# Data models for structured responses
class Flight(BaseModel):
    """Flight information"""
    flight_number: str
    airline: str
    departure: str
    arrival: str
    departure_time: str
    arrival_time: str
    price: float
    currency: str = "USD"
    available_seats: int

class Hotel(BaseModel):
    """Hotel information"""
    hotel_name: str
    location: str
    room_type: str
    price_per_night: float
    currency: str = "USD"
    available_rooms: int
    amenities: List[str]

# Underlying functions that return JSON-serializable data
def _search_flights(
    origin: str,
    destination: str,
    departure_date: str,
    passengers: int = 1
) -> List[Dict]:
    """
    Search for available flights between two cities.
    
    Args:
        origin: Departure city (e.g., "New York", "NYC")
        destination: Arrival city (e.g., "London", "LHR")
        departure_date: Date in YYYY-MM-DD format
        passengers: Number of passengers (default: 1)
    
    Returns:
        List of available flights with details (as dicts)
    """
    # Simulate flight search (in production, call real APIs)
    airlines = ["Delta", "United", "American", "British Airways"]
    
    flights = []
    for i in range(3):
        dep_time = datetime.strptime(departure_date, "%Y-%m-%d")
        dep_time = dep_time.replace(hour=random.randint(6, 20), minute=random.choice([0, 15, 30, 45]))
        arr_time = dep_time + timedelta(hours=random.randint(6, 12))
        
        flight = {
            "flight_number": f"{random.choice(['DL', 'UA', 'AA', 'BA'])}{random.randint(100, 999)}",
            "airline": random.choice(airlines),
            "departure": origin,
            "arrival": destination,
            "departure_time": dep_time.strftime("%Y-%m-%d %H:%M"),
            "arrival_time": arr_time.strftime("%Y-%m-%d %H:%M"),
            "price": round(random.uniform(300, 1200), 2),
            "currency": "USD",
            "available_seats": random.randint(5, 50)
        }
        flights.append(flight)
    
    return flights

def _check_hotel_availability(
    location: str,
    check_in: str,
    check_out: str,
    guests: int = 2
) -> List[Dict]:
    """
    Check hotel room availability in a specific location.
    
    Args:
        location: City or area name
        check_in: Check-in date in YYYY-MM-DD format
        check_out: Check-out date in YYYY-MM-DD format
        guests: Number of guests (default: 2)
    
    Returns:
        List of available hotels with details (as dicts)
    """
    # Simulate hotel search (in production, call real APIs)
    hotel_names = ["Grand Plaza Hotel", "Seaside Resort", "Downtown Suites", "Airport Inn"]
    room_types = ["Standard Room", "Deluxe Room", "Suite", "Executive Suite"]
    
    hotels = []
    for i in range(4):
        hotel = {
            "hotel_name": random.choice(hotel_names),
            "location": location,
            "room_type": random.choice(room_types),
            "price_per_night": round(random.uniform(80, 350), 2),
            "currency": "USD",
            "available_rooms": random.randint(1, 10),
            "amenities": random.sample(["WiFi", "Pool", "Gym", "Restaurant", "Spa", "Parking"], k=3)
        }
        hotels.append(hotel)
    
    return hotels

def _convert_currency(
    amount: float,
    from_currency: str,
    to_currency: str
) -> Dict:
    """
    Convert amount from one currency to another.
    
    Args:
        amount: Amount to convert
        from_currency: Source currency code (e.g., "USD", "EUR", "GBP")
        to_currency: Target currency code
    
    Returns:
        Dictionary with conversion details
    """
    # Simulate currency conversion (in production, call real exchange rate API)
    exchange_rates = {
        "USD": {"EUR": 0.85, "GBP": 0.73, "JPY": 110.0, "USD": 1.0},
        "EUR": {"USD": 1.18, "GBP": 0.86, "JPY": 129.0, "EUR": 1.0},
        "GBP": {"USD": 1.37, "EUR": 1.16, "JPY": 150.0, "GBP": 1.0},
        "JPY": {"USD": 0.0091, "EUR": 0.0078, "GBP": 0.0067, "JPY": 1.0}
    }
    
    rate = exchange_rates.get(from_currency, {}).get(to_currency, 1.0)
    converted_amount = round(amount * rate, 2)
    
    return {
        "original_amount": amount,
        "original_currency": from_currency,
        "converted_amount": converted_amount,
        "converted_currency": to_currency,
        "exchange_rate": rate,
        "timestamp": datetime.now().isoformat()
    }

def _get_server_info() -> Dict:
    """Get information about this MCP server"""
    return {
        "name": "Travel Booking Server",
        "version": "1.0.0",
        "tools": ["search_flights", "check_hotel_availability", "convert_currency"],
        "description": "MCP server for travel booking operations"
    }

# MCP tool wrappers - convert results to Pydantic models for MCP
@mcp.tool()
def search_flights(
    origin: str,
    destination: str,
    departure_date: str,
    passengers: int = 1
) -> List[Flight]:
    """
    Search for available flights between two cities.
    
    Args:
        origin: Departure city (e.g., "New York", "NYC")
        destination: Arrival city (e.g., "London", "LHR")
        departure_date: Date in YYYY-MM-DD format
        passengers: Number of passengers (default: 1)
    
    Returns:
        List of available flights with details
    """
    flights_data = _search_flights(origin, destination, departure_date, passengers)
    return [Flight(**f) for f in flights_data]

@mcp.tool()
def check_hotel_availability(
    location: str,
    check_in: str,
    check_out: str,
    guests: int = 2
) -> List[Hotel]:
    """
    Check hotel room availability in a specific location.
    
    Args:
        location: City or area name
        check_in: Check-in date in YYYY-MM-DD format
        check_out: Check-out date in YYYY-MM-DD format
        guests: Number of guests (default: 2)
    
    Returns:
        List of available hotels with details
    """
    hotels_data = _check_hotel_availability(location, check_in, check_out, guests)
    return [Hotel(**h) for h in hotels_data]

@mcp.tool()
def convert_currency(
    amount: float,
    from_currency: str,
    to_currency: str
) -> dict:
    """
    Convert amount from one currency to another.
    
    Args:
        amount: Amount to convert
        from_currency: Source currency code (e.g., "USD", "EUR", "GBP")
        to_currency: Target currency code
    
    Returns:
        Dictionary with conversion details
    """
    return _convert_currency(amount, from_currency, to_currency)

@mcp.tool()
def get_server_info() -> dict:
    """Get information about this MCP server"""
    return _get_server_info()

if __name__ == "__main__":
    # Run the server
    mcp.run()
'''

# Write the server code to a file
with open('travel_mcp_server.py', 'w') as f:
    f.write(server_code)

print("MCP Server created: travel_mcp_server.py")
print("Tools available: search_flights, check_hotel_availability, convert_currency, get_server_info")
print("\nNote: Exported _search_flights, _check_hotel_availability, _convert_currency functions")
print("      return JSON-serializable dicts for use with Azure AI Agent Framework")


MCP Server created: travel_mcp_server.py
Tools available: search_flights, check_hotel_availability, convert_currency, get_server_info

Note: Exported _search_flights, _check_hotel_availability, _convert_currency functions
      return JSON-serializable dicts for use with Azure AI Agent Framework


## Part 4: Testing the MCP Server Locally

Now let's test our MCP server locally. We'll:
1. Start the server in the background
2. Test individual tools
3. Verify the responses

In [26]:
# Test the tools by importing and calling the underlying functions
# Need to reload the module if it was already imported
import sys
if 'travel_mcp_server' in sys.modules:
    import importlib
    import travel_mcp_server
    importlib.reload(travel_mcp_server)
    from travel_mcp_server import _search_flights, _check_hotel_availability, _convert_currency, _get_server_info
else:
    from travel_mcp_server import _search_flights, _check_hotel_availability, _convert_currency, _get_server_info

# Test 1: Search for flights
print("=== Testing Flight Search ===")
flights = _search_flights(
    origin="New York",
    destination="London",
    departure_date="2025-11-15",
    passengers=2
)

for flight in flights:
    print(f"\n{flight['airline']} {flight['flight_number']}")
    print(f"  {flight['departure']} → {flight['arrival']}")
    print(f"  Departs: {flight['departure_time']}")
    print(f"  Arrives: {flight['arrival_time']}")
    print(f"  Price: ${flight['price']} ({flight['available_seats']} seats available)")


=== Testing Flight Search ===

American DL148
  New York → London
  Departs: 2025-11-15 16:00
  Arrives: 2025-11-15 22:00
  Price: $440.14 (46 seats available)

Delta BA252
  New York → London
  Departs: 2025-11-15 08:15
  Arrives: 2025-11-15 14:15
  Price: $1048.13 (34 seats available)

American DL638
  New York → London
  Departs: 2025-11-15 10:00
  Arrives: 2025-11-15 16:00
  Price: $583.09 (28 seats available)


In [27]:
# Test 2: Check hotel availability
print("\n\n=== Testing Hotel Search ===")
hotels = _check_hotel_availability(
    location="London",
    check_in="2025-11-15",
    check_out="2025-11-18",
    guests=2
)

for hotel in hotels:
    print(f"\n{hotel['hotel_name']}")
    print(f"  Location: {hotel['location']}")
    print(f"  Room Type: {hotel['room_type']}")
    print(f"  Price: ${hotel['price_per_night']}/night ({hotel['available_rooms']} rooms available)")
    print(f"  Amenities: {', '.join(hotel['amenities'])}")




=== Testing Hotel Search ===

Seaside Resort
  Location: London
  Room Type: Deluxe Room
  Price: $201.9/night (7 rooms available)
  Amenities: Pool, Gym, Restaurant

Grand Plaza Hotel
  Location: London
  Room Type: Suite
  Price: $128.94/night (7 rooms available)
  Amenities: Spa, Pool, WiFi

Grand Plaza Hotel
  Location: London
  Room Type: Standard Room
  Price: $185.24/night (4 rooms available)
  Amenities: Parking, WiFi, Spa

Grand Plaza Hotel
  Location: London
  Room Type: Deluxe Room
  Price: $216.11/night (2 rooms available)
  Amenities: Pool, Gym, Spa


In [28]:
# Test 3: Convert currency
print("\n\n=== Testing Currency Conversion ===")
conversion = _convert_currency(
    amount=1000.0,
    from_currency="USD",
    to_currency="EUR"
)

print(f"\n{conversion['original_amount']} {conversion['original_currency']} = "
      f"{conversion['converted_amount']} {conversion['converted_currency']}")
print(f"Exchange Rate: {conversion['exchange_rate']}")
print(f"Timestamp: {conversion['timestamp']}")




=== Testing Currency Conversion ===

1000.0 USD = 850.0 EUR
Exchange Rate: 0.85
Timestamp: 2025-10-13T13:48:22.642652


## Part 5: Integrating MCP Server with Azure AI Agent

Now let's build an Azure AI Agent that can use our MCP server tools. We'll use the Azure AI Agent Framework that we've been using throughout this tutorial series.

In [29]:
# Import Azure AI Agent Framework
from agent_framework import ChatAgent
from agent_framework.azure import AzureAIAgentClient
from azure.identity import DefaultAzureCredential

print("Azure AI Agent Framework imported successfully")

Azure AI Agent Framework imported successfully


In [30]:
# Create Azure AI Agent with MCP tools
from azure.identity.aio import AzureCliCredential

# Verify environment variables are set
project_endpoint = os.getenv("AZURE_AI_PROJECT_ENDPOINT")
model_deployment = os.getenv("AZURE_AI_MODEL_DEPLOYMENT_NAME")

if not project_endpoint or not model_deployment:
    print("WARNING: Missing required environment variables:")
    print("   Please set in your .env file:")
    print("   - AZURE_AI_PROJECT_ENDPOINT")
    print("   - AZURE_AI_MODEL_DEPLOYMENT_NAME")
    print("\n   See Tutorial 01b for Azure AI Foundry setup")
else:
    print("Azure AI configuration loaded")
    print(f"  Endpoint: {project_endpoint}")
    print(f"  Model: {model_deployment}")
    print("\nAgent will be created with MCP tools:")
    print("  - search_flights")
    print("  - check_hotel_availability") 
    print("  - convert_currency")


✓ Azure AI configuration loaded
  Endpoint: https://gk-agent-framework-project.services.ai.azure.com/api/projects/agentframworkProject
  Model: gpt-4.1

✓ Agent will be created with MCP tools:
  • search_flights
  • check_hotel_availability
  • convert_currency


In [33]:
# Test the agent with a simple query
import asyncio

async def test_agent_simple():
    """Test the agent with a simple travel query"""
    print("=" * 70)
    print("TEST 1: Simple Flight Search (Local Tools)")
    print("=" * 70)
    
    user_query = "I need to find flights from New York to London on November 15th for 2 passengers."
    
    print(f"\nUser: {user_query}\n")
    
    # Create and use the agent with Azure AI
    async with (
        AzureCliCredential() as credential,
        AzureAIAgentClient(
            async_credential=credential,
            # Reads AZURE_AI_PROJECT_ENDPOINT and AZURE_AI_MODEL_DEPLOYMENT_NAME from env
        ) as chat_client,
    ):
        # Enable Azure AI observability (optional but recommended)
        await chat_client.setup_azure_ai_observability()
        
        # Create agent with local Python function tools
        agent = chat_client.create_agent(
            name="TravelMCPAgent",
            instructions="""
            You are a helpful travel assistant with access to real-time travel booking tools.
            
            You can:
            - Search for flights between cities
            - Check hotel availability and pricing
            - Convert currencies for budget planning
            
            When helping users plan trips:
            1. Use the available tools to get accurate, real-time information
            2. Provide clear, organized recommendations
            3. Help with currency conversions when discussing costs
            
            Be friendly, professional, and detail-oriented.
            """,
            tools=[_search_flights, _check_hotel_availability, _convert_currency]
        )
        
        # Run the agent
        response = await agent.run(user_query)
        
        print(f"Agent: {response.text}\n")
    
    print("=" * 70)
    print("Note: This uses LOCAL Python functions as tools")
    print("      See Tutorial 16 for deploying as a remote MCP server")

# Run the test only if environment is configured
if project_endpoint and model_deployment:
    await test_agent_simple()
else:
    print("WARNING: Please configure Azure AI environment variables first")
    print("   See cell above for required configuration")


TEST 1: Simple Flight Search (Local Tools)

User: I need to find flights from New York to London on November 15th for 2 passengers.



[2025-10-13 14:10:48 - /Users/gk/AIFoundryProjects/agent-framework-tutorials/venv/lib/python3.12/site-packages/azure/monitor/opentelemetry/exporter/export/_base.py:427 - WARNING] Retrying due to server request error: <urllib3.connection.HTTPSConnection object at 0x117402750>: Failed to establish a new connection: [Errno 61] Connection refused.
[2025-10-13 14:10:52 - /Users/gk/AIFoundryProjects/agent-framework-tutorials/venv/lib/python3.12/site-packages/agent_framework/_clients.py:609 - WARNING] When conversation_id is set, store must be True for service-managed threads. Automatically setting store=True.
[2025-10-13 14:10:52 - /Users/gk/AIFoundryProjects/agent-framework-tutorials/venv/lib/python3.12/site-packages/agent_framework/_clients.py:609 - WARNING] When conversation_id is set, store must be True for service-managed threads. Automatically setting store=True.


Agent: Here are some available flights from New York to London on November 15th for 2 passengers:

1. Delta Flight AA223
   - Departure: 09:30 AM
   - Arrival: 04:30 PM
   - Price: $861.34 per person
   - Seats Available: 16

2. Delta Flight BA229
   - Departure: 06:45 AM
   - Arrival: 01:45 PM
   - Price: $1,066.68 per person
   - Seats Available: 10

3. United Flight UA634
   - Departure: 08:30 PM
   - Arrival: 08:30 AM (next day)
   - Price: $953.97 per person
   - Seats Available: 18

If you’d like more details or want to proceed with booking, please let me know your preferences!

Note: This uses LOCAL Python functions as tools
      See Tutorial 16 for deploying as a remote MCP server
Note: This uses LOCAL Python functions as tools
      See Tutorial 16 for deploying as a remote MCP server


In [34]:
# Test the agent with a complex multi-tool query
async def test_agent_complex():
    """Test the agent with a complex query requiring multiple tools"""
    print("\n" + "=" * 70)
    print("TEST 2: Complex Multi-Tool Query (Local Tools)")
    print("=" * 70)
    
    user_query = """
    I'm planning a trip to London from New York. Can you help me with:
    1. Flights for November 15th for 2 passengers
    2. Hotels for 3 nights (Nov 15-18)
    3. Total cost estimate in Euros if my budget is $2000 USD
    """
    
    print(f"\nUser: {user_query}\n")
    
    # Create and use the agent with Azure AI
    async with (
        AzureCliCredential() as credential,
        AzureAIAgentClient(
            async_credential=credential,
        ) as chat_client,
    ):
        # Enable Azure AI observability
        await chat_client.setup_azure_ai_observability()
        
        # Create agent with local Python function tools
        agent = chat_client.create_agent(
            name="TravelMCPAgent",
            instructions="""
            You are a helpful travel assistant with access to real-time travel booking tools.
            Use multiple tools as needed to provide comprehensive travel planning assistance.
            """,
            tools=[_search_flights, _check_hotel_availability, _convert_currency]
        )
        
        # Run the agent - it will automatically use multiple tools as needed
        response = await agent.run(user_query)
        
        print(f"Agent: {response.text}\n")
    
    print("=" * 70)
    print("Note: This uses LOCAL Python functions as tools")
    print("      Tools run in the same process as the agent")

# Run the complex test only if environment is configured
if project_endpoint and model_deployment:
    await test_agent_complex()
else:
    print("WARNING: Please configure Azure AI environment variables first")



TEST 2: Complex Multi-Tool Query (Local Tools)

User: 
    I'm planning a trip to London from New York. Can you help me with:
    1. Flights for November 15th for 2 passengers
    2. Hotels for 3 nights (Nov 15-18)
    3. Total cost estimate in Euros if my budget is $2000 USD
    



[2025-10-13 14:15:42 - /Users/gk/AIFoundryProjects/agent-framework-tutorials/venv/lib/python3.12/site-packages/agent_framework/_clients.py:609 - WARNING] When conversation_id is set, store must be True for service-managed threads. Automatically setting store=True.


Agent: Here’s a summary for your London trip (Nov 15–18) from New York for 2 passengers:

1. Flights (Direct, Nov 15th):
   - British Airways (AA929): 20:15–06:15 (Nov 16), $431.68 per person (Total: $863.36)
   - United (BA575): 19:15–02:15 (Nov 16), $514.39 per person (Total: $1028.78)
   - American Airlines (AA963): 06:00–18:00, $1106.32 per person (Total: $2212.64)

2. Hotels in London (Nov 15–18, prices per night):
   - Downtown Suites (Executive Suite): $83.32/night (Total: $249.96 for 3 nights)
   - Seaside Resort (Standard Room): $150.85/night (Total: $452.55 for 3 nights)
   - Grand Plaza Hotel (Deluxe Room): $160.51/night (Total: $481.53 for 3 nights)
   - Downtown Suites (Suite): $147.79/night (Total: $443.37 for 3 nights)

3. Budget in Euros:
   - $2000 USD ≈ €1700 EUR (exchange rate 0.85)

Sample cost estimate (cheapest options):
- British Airways flight (2 passengers): $863.36
- Downtown Suites (Executive Suite, 3 nights): $249.96
- Total: $1113.32 USD ≈ €946 EUR

This le

## Understanding Local vs Hosted MCP Tools

### What We Just Built

In this tutorial, we used **local Python functions as tools**:
```python
# Tools are Python functions in the same process
agent = chat_client.create_agent(
    tools=[_search_flights, _check_hotel_availability, _convert_currency]
)
```

**This approach is perfect for:**
- Development and testing
- Quick prototyping
- Single-agent applications
- Learning MCP concepts

### What's Next: Hosted MCP Servers

In **Tutorial 16**, you'll deploy this MCP server to Azure and use **HostedMCPTool**:
```python
# Tools are hosted on a remote MCP server
agent = chat_client.create_agent(
    tools=HostedMCPTool(
        name="Travel Booking MCP Server",
        url="https://your-app.azurecontainerapps.io",
        approval_mode="never_require"
    )
)
```

**Hosted MCP is better for:**
- Production deployments
- Multi-agent systems sharing tools
- Tools requiring isolation or independent scaling
- Enterprise applications with security requirements

### Key Difference

| Aspect | Local Tools (Tutorial 15) | Hosted MCP (Tutorial 16) |
|--------|---------------------------|--------------------------|
| **Location** | Same process as agent | Remote HTTP server |
| **Performance** | Fast (no network) | Network latency |
| **Scaling** | Scales with agent | Independent scaling |
| **Sharing** | One agent only | Multiple agents |
| **Deployment** | Simple | Requires infrastructure |
| **Use Case** | Development/Testing | Production |


## Part 6: Best Practices for MCP Server Development

### Critical Lessons Learned

**1. Dual Function Pattern (Required for Agent Integration)**
```python
# Underscore functions return JSON-serializable dicts (for agents)
def _search_flights(...) -> List[Dict]:
    return [{"flight_number": "AA123", ...}]

# MCP wrappers convert to Pydantic models (for MCP server)
@mcp.tool()
def search_flights(...) -> List[Flight]:
    return [Flight(**f) for f in _search_flights(...)]
```

**2. Package Management**
- Don't pin FastMCP versions - use `pip install fastmcp` without constraints
- Let pip auto-resolve dependencies (FastMCP evolves rapidly)

**3. Environment & Security**
- Store credentials in `.env` files
- Validate environment variables before agent creation
- Never hardcode sensitive values

**4. Testing & Error Handling**
- Test underscore functions locally first
- Test with agents to verify JSON serialization
- Wrap tool functions in try-except blocks
- Return structured error responses

**5. Production Considerations**
- Make tools async for I/O operations
- Implement rate limiting and timeouts
- Use caching for frequently requested data
- Monitor tool invocations and error rates

## Summary and Key Takeaways

### What You've Learned

1. **Model Context Protocol (MCP)**: A standardized way to connect AI agents to external tools and data sources

2. **FastMCP Framework**: Python framework for building MCP servers with:
   - Decorator-based tool definitions
   - Automatic schema generation
   - Built-in validation

3. **Tool Development**: How to create MCP tools with:
   - Clear function signatures
   - Pydantic models for structured data
   - Proper documentation

4. **Agent Integration**: How to connect MCP servers to AI agents using:
   - OpenAI function calling
   - Tool execution handlers
   - Multi-turn conversations

### Next Steps

In the next tutorial (**16_deploy_fastmcp_to_aca.ipynb**), you'll learn how to:
- Dockerize your MCP server
- Deploy to Azure Container Apps
- Configure environment variables and scaling
- Monitor production MCP servers

### Additional Resources

- **FastMCP Documentation**: https://github.com/jlowin/fastmcp
- **MCP Specification**: https://modelcontextprotocol.io
- **Azure Container Apps**: https://learn.microsoft.com/azure/container-apps/

---

**Congratulations!** You've built your first MCP server and integrated it with an AI agent. This is the foundation for building scalable, enterprise-grade AI agent architectures.